In [171]:
import json
import re
import pandas as pd
import xlwings as xw


In [172]:
with open("all_commands.json", "r") as file:
    all_commands = json.load(file)


In [173]:
commands = []

for c in all_commands:
    if "Available Commands" not in c:
        commands.append(c)


In [174]:
count = 0
for i in commands:
    all_flags = []
    all_flags.extend(i["Flags"])
    del i["Flags"]
    if "Global Flags" in i:
        all_flags.extend(i["Global Flags"])
        del i["Global Flags"]

    if all_flags != []:
        i["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [175]:
for c in commands:
    # c["flags_parsed"] = [f for f in c["flags_parsed"] if f["long"] != "help"]
    c["Usage"] = [
        l.replace("[flags]", "").replace(c["command"], "").strip() for l in c["Usage"]
    ]
    c['Description'] = " ".join(c['Description'])[:200]

In [177]:
for c in commands:
    clean_string = c['Usage'][0].split("(")[0].split("--")[0]
    args = re.findall(r'(?:\[|\<)(\w.+?)(?:\]|\>)', clean_string)
    c['args'] = []
    for p in args:
        p = re.sub(r'[^A-Za-z0-9]', '_', p)
        c['args'].append(p)


In [178]:
# df = pd.DataFrame(commands)
# pd.json_normalize(df['flags_parsed'])

In [179]:
# df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
# df['command'] = df['command'].apply(lambda x: str(x))
# df2 = pd.json_normalize(df['flags_parsed'])

# df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
# xw.view(df3)


In [180]:
# for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
# xw.view(for_excel)


In [181]:
# for i in all_commands:
#     print(i["command"], "\n", i["flags_parsed"])


In [182]:
types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}
lines = []

for c in commands:
    lines.append(f"\n# {c['Description']}\nextern '{c['command']}' [\n")
    for p in c['args']:
        lines.append(f'\t{p}: string\n')
    # print(f"\n# {c['Description']}\nextern '{c['command']}' [")
    for f in c["flags_parsed"]:
        if "." not in f["long"]:
            long = f["long"]
            short = "(-" + f["short"] + ")" if f["short"] else ""
            f1 = types_nu_dict.get(f["format"], f["format"])
            format = ": " + f1 if f1 else ""
            description = f["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
            # print(f"\t--{long}{short}{format}\t\t# {description}")
    lines.append("]")
    # print("]")

with open("cyber-autocompletions.nu", "w") as file:
    file.writelines(lines)


In [88]:
import pandas as pd


In [ ]:
df = pd.DataFrame(all_commands).sort_values("command", ascending=False)


In [ ]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()


In [ ]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))


In [ ]:
_list = []

for _file in os.scandir(
    "/Users/user/apps-files/github/cyber_completitions/json-files/"
):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))


In [ ]:
[i for i in _list if i != []]


In [ ]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df


In [ ]:
df["com1"] = "cyber " + df["_command_root"] + " " + df["_command"]
df


In [ ]:
_l2 = df.loc[:, ["_description", "com1"]].to_dict("records")
_l2


In [ ]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')


In [ ]:
for i in json.loads(content):
    print("\t", i["_command"], "\t\t#", i["_description"])
